### Import Required Packages and Set Options

#### Import Base Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rankfm.rankfm import RankFM
import hit_rates as hr
import numpy as np

# Things to do
* Take auxiliary data into account, such as 
    * age, gender (probably class imbalance) (20-40, 40-60, 60-80)
    * booking date (month/day of week (dow)) (1-12/1-7)
    * flight date (month/dow) (1-12/1-7)
    * family size (1-7)
    * trip length (nb days): 1-9, 10+. (should plot data. Avoid class imbalance)
* Number of categories: 3 + 12 + 7 + 12 + 7 + 7 + 10 = 65
    * The number of combinations: 3 * 12 * 12 * 7 * 7 * 7 * 10 = 3 * 144 * 343 * 10 = 1,481,760
    * We have about 1,000,000 records. So there are more combinations than we have data. 
        Is that good or bad? 
        
## Regularization parameters
Currently, there are two regularization parameters; $\alpha$ for the weights corresponding to the user-item matrix, 
and $\beta$ for the item and user attributes. Ideally, $\beta$ should be broken up into $\beta_i$ and $\beta_u$ for item nad user attributes, respectively. In Field-Aware Machine Factorization, each field (collection of features) has its own regularizing parameters (as far as I understand it). 

In [3]:
# Optimum 25
samples = np.arange(5, 51, 10).astype('int32')
# Optimimum 20
nb_factors = [5,10,20,30]
# Optimum: 0.01 or 0.05
alphas = [0.001, 0.01, 0.05, 0.1, 0.5, 1.]
betas = [1000., 0.01, 0.05 ,0.1]
nothing = [1,1,1,1,1]

In [50]:
# Optimum parameters when keep_nb_users=2000
# hit rate: 0.27
max_samples = 5
factors = 20
keep_nb_users = 2000
alpha = 0.01
beta = 0.01

# When filter_previous==True, the results are much more sensitive to alpha. 
# it would be great if alpha could be learned during optimization. 
# hit rate: 0.76 (baseline based on most popular: 0.45) (averaged over 5 runs)
alpha = 0.01

In [56]:
# max_samples=500 creates problem for 'warp', but not for 'bpr'. Or vce-versa. What is the difference? And Why?
# for max_samples in samples:
# for factors in nb_factors:
# for alpha in alphas:
# for beta in betas:
for n in nothing:
    model = RankFM(factors=factors,
                   loss='warp', max_samples=int(max_samples), alpha=alpha, beta=beta,
                   learning_rate=0.1, learning_schedule='invscaling')

    results = hr.evaluate_hit_rate(model, '2016', '2017', filter_previous=True, 
                    keep_nb_users=keep_nb_users, verbose=False, nb_epochs=30)
    results['max_samples'] = max_samples
    print(max_samples, results['model_hrt'])
#     print(results)
#     break

Evaluate_hit_rate: years: 2016, 2017
5 {0: 0.2603053435114504}
Evaluate_hit_rate: years: 2016, 2017
5 {0: 0.24817518248175183}
Evaluate_hit_rate: years: 2016, 2017
5 {0: 0.29037149355572406}
Evaluate_hit_rate: years: 2016, 2017
5 {0: 0.2604087812263437}
Evaluate_hit_rate: years: 2016, 2017
5 {0: 0.26410835214446954}


In [168]:
# max_samples=500 creates problem for 'warp', but not for 'bpr'. Or vce-versa. What is the difference? And Why?
# for max_samples in samples:
# for factors in nb_factors:
    model = RankFM(factors=factors,
                   loss='warp', max_samples=int(max_samples), alpha=0.01, 
                   learning_rate=0.1, learning_schedule='invscaling')

    results = hr.evaluate_hit_rate(model, '2016', '2017', filter_previous=True, 
                    keep_nb_users=20000, verbose=False, nb_epochs=30)
    results['max_samples'] = max_samples
    print(max_samples, results['model_hrt'])
#     print(results)
#     break

IndentationError: unexpected indent (<ipython-input-168-5431bdd64f80>, line 4)

In [ ]:
adfabdadfdfaf

In [76]:
results = hr.evaluate_hit_rate(model, '2016', '2017', filter_previous=True, keep_nb_users=20000,
                    verbose=False)

Evaluate_hit_rate: years: 2016, 2017


In [77]:
results

{'keep_nb_users': 20000,
 'filter_previous': True,
 'year1': '2016',
 'year2': '2017',
 'nb_epochs': '2017',
 'seed': None,
 'sparsity_all': 0.9548386510497471,
 'sparsity': 0.9547455128205128,
 'topN=k': 3,
 'most_popular': product_id
 BOG    2769
 PTY    2757
 MIA    2508
 Name: user_id, dtype: int64,
 'base_hrt': 0.45251025157929736,
 'base_pre': 0.17231888876574677,
 'base_rec': 0.2305987722785726,
 'model_rnk': 0.19377669927453803,
 'model_pre': 0.1046232400596547,
 'model_rec': 0.09971190688678604,
 'model_hrt': [{0: 0.27754606809736865,
   1: 0.03480700690073557,
   2: 0.0015166451808599378},
  {0: 0.27754606809736865, 1: 0.03480700690073557, 2: 0.0015166451808599378},
  {0: 0.27754606809736865, 1: 0.03480700690073557, 2: 0.0015166451808599378}]}

In [66]:
hr.evaluate_hit_rate('2016', '2017', keep_nb_users=None, 
                     nb_epochs=100, verbose=True, 
                     filter_previous=True)

datetime.now:  2022-05-21 19:06:33.165236
Evaluate_hit_rate: years: 2016, 2017
full interaction data sparsity: 95.48
sample users: 31028
sample items: 78
sample interactions: (137610, 3)
sample interaction data sparsity: 95.48
total shape: (137610, 2)
train shape: (62971, 2)
valid shape: (74639, 2)

train weights shape: (62971,)
valid weights shape: (74639,)

nb train users: 23554
nb valid users: 28003
nb cold-start users: 7474

train items: 76
valid items: 75
number of cold-start items: 2
cold start items:  {'DEN', 'MDZ'}

training epoch: 0
log likelihood: -38955.76953125

training epoch: 1
log likelihood: -39071.19921875

training epoch: 2
log likelihood: -38878.328125

training epoch: 3
log likelihood: -38614.73828125

training epoch: 4
log likelihood: -38034.671875

training epoch: 5
log likelihood: -37369.76953125

training epoch: 6
log likelihood: -36476.76953125

training epoch: 7
log likelihood: -35462.21875

training epoch: 8
log likelihood: -34421.19921875

training epoch: 9


In [41]:
hr.evaluate_hit_rate('2020', '2021', nb_epochs=30, filter_previous=True, keep_nb_users=1000, verbose=False)

datetime.now:  2022-05-21 16:12:05.478385
Evaluate_hit_rate: years: 2020, 2021
full interaction data sparsity: 97.64
number of test users: 745
baseline hit rate: 0.416
baseline precision: 0.150
baseline recall: 0.300
model hit rate: 0.686
model reciprocal rank: 0.6
model precision: 0.263
model recall: 0.509
model hit rate (average nb hits > 0): 0.1703056768558952
model hit rate (average nb hits > 1): 0.008733624454148471
model hit rate (average nb hits > 2): 0.002183406113537118
model reciprocal rank: 0.6
model precision: 0.263
model recall: 0.509


In [47]:
hr.evaluate_hit_rate('2021', '2020', nb_epochs=30, filter_previous=True, keep_nb_users=1000, verbose=False)

datetime.now:  2022-05-21 16:13:38.947386
Evaluate_hit_rate: years: 2021, 2020
full interaction data sparsity: 97.64
number of test users: 730
baseline hit rate: 0.326
baseline precision: 0.111
baseline recall: 0.255
model hit rate: 0.68
model reciprocal rank: 0.565
model precision: 0.257
model recall: 0.56
model hit rate (average nb hits > 0): 0.12612612612612611
model hit rate (average nb hits > 1): 0.0022522522522522522
model hit rate (average nb hits > 2): 0.0
model reciprocal rank: 0.565
model precision: 0.257
model recall: 0.56
